In [1]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [7]:
function bosonicOperators(cutoff::Int, elt=Float64)
    creadat = zeros(elt, cutoff + 1, cutoff + 1)

    for i in 1:cutoff
        creadat[i + 1, i] = sqrt(i)
    end

    a⁺ = TensorMap(creadat, ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    a⁻ = TensorMap(collect(creadat'), ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    return (a⁺, a⁻)
end

bosonicOperators (generic function with 2 methods)

In [8]:
# define Hamiltionian for infinite basis with chemical potential
function BoseHubbardHamiltonian(t, U, μ, L, M)

    a⁺, a⁻ = bosonicOperators(M) # bosonic operators
    n_op = a⁺ * a⁻

    H_i = TensorMap(zeros, Float64, (ℝ^(M+1))^2 ← (ℝ^(M+1))^2)
    H_i += -t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺) + U/2 * n_op * (n_op - id(ℝ^(M+1))) ⊗ id(ℝ^(M+1))
    H_i += -μ * n_op ⊗ id(ℝ^(M+1))

    return @mpoham sum(H_i{i,i+1} for i in (1:L))
end

BoseHubbardHamiltonian (generic function with 1 method)

In [60]:
H = BoseHubbardHamiltonian(0.001, 4, 0, 128, 5)

MPOHamiltonian{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}(MPSKit.SparseMPOSlice{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}[[TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^1)):
[:, :, 1, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  1.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  1.0  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  1.0  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0
 TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^3)):
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  -0.08276058886023684  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  -0.24828176658071027  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  -0.4965635331614207  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  -0.8276058886023683

[:, :, 1, 2] =
 -6.63392351284039e-26  0.25819888974698346  0.0 

In [61]:
state = FiniteMPS(rand, Float64, 128, ℝ^(5+1), ℝ^10)

FiniteMPS{TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}, TensorMap{CartesianSpace, 1, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}(Union{Missing, TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}[TensorMap((ℝ^1 ⊗ ℝ^6) ← ProductSpace(ℝ^6)):
[:, :, 1] =
 0.27719460594199585  0.36212867888171446  …  0.6344346840926411

[:, :, 2] =
 0.12953047608333138  0.11118007979253641  …  -0.035392227271037746

[:, :, 3] =
 0.813535582696781  -0.3299747846611471  …  0.18383740942169075

[:, :, 4] =
 -0.4123556180129436  -0.11987089892014992  …  0.35298433629070713

[:, :, 5] =
 0.2555653151332217  0.6267669581318346  …  -0.5451325902241332

[:, :, 6] =
 -0.09588574389614106  0.5834488829904305  …  0.3750684553186572
, TensorMap((ℝ^6 ⊗ ℝ^6) ← ProductSpace(ℝ^10)):
[:, :, 1] =
 0.37722841856388967   0.3444595092670331     …  0.2748416699738172
 0.03652988691436269   0.06705582882662685       0.028961753383085594
 0.0682181520508873    0.01494337

In [62]:
(groundstate,_) = find_groundstate(state, H, DMRG(tol=1e-10, maxiter=1000));

┌ Info: Iteraton 0 error 0.011268071579956936
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.0025520967947719555
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 0.0008355898336223493
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 0.0004795864864953209
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 0.0001948604625697638
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


...

┌ Info: Iteraton 152 error 1.4496260944657593e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 153 error 1.3068870811257473e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 154 error 1.1782644897469865e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 155 error 1.062286351195254e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 156 error 9.578148025878402e-11
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [63]:
E0 = sum(expectation_value(groundstate, H))

-0.082341965287215

In [64]:
a⁺, a⁻ = bosonicOperators(5) # bosonic operators
n_op = a⁺ * a⁻

TensorMap(ProductSpace(ℝ^6) ← ProductSpace(ℝ^6)):
 0.0  0.0  0.0                 0.0                 0.0  0.0
 0.0  1.0  0.0                 0.0                 0.0  0.0
 0.0  0.0  2.0000000000000004  0.0                 0.0  0.0
 0.0  0.0  0.0                 2.9999999999999996  0.0  0.0
 0.0  0.0  0.0                 0.0                 4.0  0.0
 0.0  0.0  0.0                 0.0                 0.0  5.000000000000001


In [65]:
N_b = expectation_value(groundstate, n_op)

128-element Vector{Float64}:
 0.5004873708269194
 0.5001306178201889
 0.5007201058871028
 0.5000615613705195
 0.5007534938473447
 0.5000347977931483
 0.500769575674442
 0.5000162494183553
 0.5007796746266397
 0.5000016361377291
 ⋮
 0.48728336427440727
 0.51373367192975
 0.48489885144588885
 0.5166374224420844
 0.4815155599650094
 0.5204866775560676
 0.4791613435595867
 0.5016450611454196
 4.293024362964129

In [66]:
sum(N_b)

67.81505633991975